In [1]:
print 'load libraries...'

load libraries...


In [2]:
# Aim: 

In [3]:
# Add path in order to load utility functions
import sys
sys.path.append(r"N:\18. Hot Spot\Custom Modules\Utils")
sys.path.append(r"N:\18. Hot Spot\Custom Modules\ETLMonthly")

In [4]:
# setup environment and import libraries
import os
import re
import datetime
import time
import getpass
import pandas as pd
import numpy as np
from datetime import datetime, date
from datetime import timedelta
import calendar as cd
from collections import OrderedDict
path=r'C:\ProgramData\Anaconda2\instantclient_12_2'
os.environ['PATH']=path
import cx_Oracle
import file_utils
import db_utils
import config
#set working directory
os.chdir(config.working_dir)

In [5]:
pd.set_option('display.max_columns', 500)

In [6]:
# db credentials authentication

In [7]:
print 'connect to database...'
conn_string = db_utils.db_connect(config.db_username,config.db_pwd)

connect to database...
Testing Connection...
Connection Succesful!


In [8]:
mtch = re.match('.*@', conn_string)
un, pwd = mtch.group().rstrip('@').split('/')

In [9]:
con = cx_Oracle.connect(conn_string)
df_cs_retail_loan_cust = pd.read_sql_query(r'select bra_code, cus_num from cs_retail_loan_cust', con)
con.close()
len(df_cs_retail_loan_cust)

154724

In [10]:
print 'get POS trxns of retail loan customers...'

db_utils.create_table_from_selectquery(
'cs_retail_loan_cust_pos_trxns',
'''
select bra_code, cus_num, cur_code, led_code, sub_acct_code, is_bad, trxn_hour, tran_amount_rsp,
rsp_code_rsp, merchant_type, card_acceptor_id_code, trxn_day, trxn_date,
(case when trxn_hour between 0 and 5 then '0-5'
when trxn_hour between 6 and 11 then '6-11'
when trxn_hour between 12 and 16 then '12-16'
when trxn_hour between 17 and 20 then '17-20'
when trxn_hour between 21 and 23 then '21-23' end) as trxn_hour_period

from
(

select a.*, b.*, to_number(to_char(to_timestamp(datetime_req),'HH24')) trxn_hour,
to_char(to_timestamp(datetime_req),'D') trxn_day, trunc(to_timestamp(datetime_req)) trxn_date
from cs_retail_loan_cust a
inner join
(select * from SRC_FEP_LOCAL_POS_TRANX_2018 where to_timestamp(datetime_req) between '01jan2018' and '30dec2018') b
on a.bra_code||'0'||a.cus_num = substr(from_account_id,1,10)
and to_timestamp(b.datetime_req) between trunc(add_months(a.date_open, -12),'month')
and trunc(add_months(a.date_open, -1),'month')

union all

select a.*, b.*, to_number(to_char(to_timestamp(datetime_req),'HH24'))trxn_hour, 
to_char(to_timestamp(datetime_req),'D') trxn_day, trunc(to_timestamp(datetime_req)) trxn_date
from cs_retail_loan_cust a
inner join
(select * from SRC_FEP_LOCAL_POS_TRANX_2019 where to_timestamp(datetime_req) >= '01jan2019') b
on a.bra_code||'0'||a.cus_num = substr(from_account_id,1,10)
and to_timestamp(b.datetime_req) between trunc(add_months(a.date_open, -12),'month')
and trunc(add_months(a.date_open, -1),'month')
)
''',
conn_string, drop_if_exists=True)

get POS trxns of retail loan customers...
Creating New Table...
New table records:  3992025


In [11]:
con = cx_Oracle.connect(conn_string)
pos_trxns_info = pd.read_sql_query('''
select bra_code, cus_num, sum(tran_amount_rsp) pos_tra_amt, count(1) pos_tra_count,
count(case when trxn_hour_period='0-5' then 1 end ) pos_tra_count_0_5,
count(distinct case when trxn_hour_period='0-5' then trxn_date end ) pos_tra_day_count_0_5,
count(case when trxn_hour_period='6-11' then 1 end ) pos_tra_count_6_11,
count(distinct case when trxn_hour_period='6-11' then trxn_date end ) pos_tra_day_count_6_11,
count(case when trxn_hour_period='12-16' then 1 end ) pos_tra_count_12_16,
count(distinct case when trxn_hour_period='12-16' then trxn_date end ) pos_tra_day_count_12_16,
count(case when trxn_hour_period='17-20' then 1 end ) pos_tra_count_17_20,
count(distinct case when trxn_hour_period='17-20' then trxn_date end ) pos_tra_day_count_17_20,
count(case when trxn_hour_period='21-23' then 1 end ) pos_tra_count_21_23,
count(distinct case when trxn_hour_period='21-23' then trxn_date end ) pos_tra_day_count_21_23
from data_analyst.cs_retail_loan_cust_pos_trxns
group by bra_code, cus_num
''', con)
con.close()

In [14]:
# df_cs_retail_loan_cust.merge(pos_trxns_info, on=['BRA_CODE','CUS_NUM'],
#                              how='left').fillna(0).to_csv('cs_pos_trxns_info.csv', index=False)

In [15]:
pos_trxns_info.to_csv('cs_pos_trxns_info.csv', index=False)